In [65]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import FloatSlider, HBox, VBox, interactive_output
import ipywidgets as widgets

def plot_efficient_frontier(
        mu1, mu2, mu3, var1, var2, var3, corr12, corr13, corr23,
        min_alpha, max_alpha, cml_slope,
        x_min, x_max, y_min, y_max,
        cml_slope_multiplier,
        cml_alpha
    ):
    # Means (Expected returns)
    mu = np.array([mu1, mu2, mu3])
    
    # Variances
    var = np.array([var1, var2, var3])
    
    # Covariance matrix
    cov = np.zeros((3, 3))
    np.fill_diagonal(cov, var)
    cov[0, 1] = corr12 * np.sqrt(var[0] * var[1])
    cov[1, 0] = cov[0, 1]
    cov[0, 2] = corr13 * np.sqrt(var[0] * var[2])
    cov[2, 0] = cov[0, 2]
    cov[1, 2] = corr23 * np.sqrt(var[1] * var[2])
    cov[2, 1] = cov[1, 2]
    
    # Inverse of covariance matrix
    try:
        cov_inv = np.linalg.inv(cov)
    except np.linalg.LinAlgError:
        print("Covariance matrix is singular and cannot be inverted.")
        return
    
    ones = np.ones(len(mu))
    
    # Recalculate the tangency portfolio weights
    weights_tangency = cov_inv @ mu
    weights_tangency /= np.sum(weights_tangency)
    
    # Recalculate the GMV portfolio weights
    weights_gmv = cov_inv @ ones
    weights_gmv /= np.sum(weights_gmv)
    
    # Generate efficient frontier by combining tangency and GMV portfolios
    alphas = np.linspace(min_alpha, max_alpha, 10_000)  # Allows for extrapolation beyond the two portfolios
    returns = []
    variances = []
    
    for alpha in alphas:
        weights = alpha * weights_tangency + (1 - alpha) * weights_gmv
        port_return = np.dot(weights, mu)
        port_variance = np.dot(weights.T, np.dot(cov, weights))
        returns.append(port_return)
        variances.append(port_variance)
    
    returns = np.array(returns)
    variances = np.array(variances)
    
    # Risk-free rate (assuming zero for excess return)
    risk_free_rate = 0
    
    # Calculate Sharpe Ratios
    with np.errstate(divide='ignore', invalid='ignore'):
        sharpe_ratios = (returns - risk_free_rate) / np.sqrt(variances)
    
    # Identify tangency portfolio (maximum Sharpe Ratio)
    max_sharpe_idx = np.argmax(sharpe_ratios)
    max_sharpe_var = variances[max_sharpe_idx]
    max_sharpe_ret = returns[max_sharpe_idx]
    
    # Plot Efficient Frontier
    plt.figure(figsize=(8, 6))
    plt.plot(variances, returns - risk_free_rate, 'b--', label='Efficient Frontier')
    
    # Plot Capital Market Line (CML)
    # CML is the line from (0, 0) to the tangency portfolio, extended
    cml_x = np.linspace(0, x_max, 500)
    cml_y = cml_slope * cml_slope_multiplier * cml_x
    plt.plot(cml_x, cml_y, color='r', label=f'MV + RF Opt. {(cml_slope * cml_slope_multiplier):.2f}', alpha=cml_alpha)
    plt.plot(cml_x, -cml_y, color='r', alpha=cml_alpha)
    
    # Mark Tangency and GMV Portfolios
    # plt.scatter([max_sharpe_var], [max_sharpe_ret - risk_free_rate], color='g', label='Tangency Portfolio')
    gmv_return = np.dot(weights_gmv, mu)
    gmv_variance = np.dot(weights_gmv.T, np.dot(cov, weights_gmv))
    plt.scatter([gmv_variance], [gmv_return - risk_free_rate], color='orange', label=f'GMV Portfolio ({gmv_variance:.1%}, {gmv_return:.1%})')

    plt.axhline(y=0, linestyle='--', color='black', alpha=0.5)
    plt.scatter([var1], [mu1], color='purple', label=f'Asset 1 ({var1:.1%}, {mu1:.1%})')
    plt.scatter([var2], [mu2], color='green', label=f'Asset 2 ({var2:.1%}, {mu2:.1%})')
    plt.scatter([var3], [mu3], color='yellow', label=f'Asset 3 ({var3:.1%}, {mu3:.1%})')
    
    # Set x and y limits
    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)
    
    # Labels and Title
    weights_tangency_str = ", ".join([f"{w:.0%}" for w in weights_tangency])
    weights_gmv_str = ", ".join([f"{w:.0%}" for w in weights_gmv])
    plt.xlabel(f'Variance; Weights Tangency: {weights_tangency_str}; Weights GMV: {weights_gmv_str}')
    plt.ylabel('Excess Return (Expected Return - Risk-Free Rate)')
    plt.title('Efficient Frontier with Capital Market Line')
    plt.legend()
    plt.grid(True)
    plt.show()

# Create interactive widgets for parameters
mu1_slider = FloatSlider(value=0.04, min=-0.20, max=0.20, step=0.01, description='Mean 1')
mu2_slider = FloatSlider(value=0.15, min=-0.20, max=0.20, step=0.01, description='Mean 2')
mu3_slider = FloatSlider(value=0.05, min=-0.20, max=0.20, step=0.01, description='Mean 3')

var1_slider = FloatSlider(value=0.02, min=0.0001, max=0.10, step=0.0001, description='Var 1')
var2_slider = FloatSlider(value=0.03, min=0.0001, max=0.10, step=0.0001, description='Var 2')
var3_slider = FloatSlider(value=0.015, min=0.0001, max=0.10, step=0.0001, description='Var 3')

corr12_slider = FloatSlider(value=0.2, min=-1, max=1, step=0.05, description='Corr 1-2')
corr13_slider = FloatSlider(value=0.1, min=-1, max=1, step=0.05, description='Corr 1-3')
corr23_slider = FloatSlider(value=0.3, min=-1, max=1, step=0.05, description='Corr 2-3')

min_alpha_slider = FloatSlider(value=-5, min=-10, max=0, step=1, description='Min Delta')
max_alpha_slider = FloatSlider(value=10, min=1, max=20, step=1, description='Max Delta')
cml_slope_slider = FloatSlider(value=8.4, min=-10, max=10, step=0.1, description='MV RF Line')
cml_slope_multiplier_slider = FloatSlider(value=1, min=0.1, max=3, step=0.1, description='Mult')

x_min_slider = FloatSlider(value=0, min=0, max=0.1, step=0.005, description='X-axis Min')
x_max_slider = FloatSlider(value=0.05, min=0.005, max=0.2, step=0.001, description='X-axis Max')
y_min_slider = FloatSlider(value=0, min=-1, max=0, step=0.01, description='Y-axis Min')
y_max_slider = FloatSlider(value=0.24, min=0, max=1, step=0.01, description='Y-axis Max')
cml_alpha_slider = FloatSlider(value=0, min=0, max=1, step=1, description='MV RF On')

# Organize sliders into groups for better layout
sliders_column1 = VBox([mu1_slider, mu2_slider, mu3_slider, var1_slider, var2_slider, var3_slider, corr12_slider, corr13_slider, corr23_slider, min_alpha_slider, max_alpha_slider])
sliders_column2 = VBox([cml_slope_slider, cml_slope_multiplier_slider, x_min_slider, x_max_slider, y_min_slider, y_max_slider, cml_alpha_slider])

controls = HBox([sliders_column1, sliders_column2])

# Use interactive_output to connect the function with the widgets
out = interactive_output(plot_efficient_frontier, {
    'mu1': mu1_slider,
    'mu2': mu2_slider,
    'mu3': mu3_slider,
    'var1': var1_slider,
    'var2': var2_slider,
    'var3': var3_slider,
    'corr12': corr12_slider,
    'corr13': corr13_slider,
    'corr23': corr23_slider,
    'min_alpha': min_alpha_slider,
    'max_alpha': max_alpha_slider,
    'cml_slope': cml_slope_slider,
    'cml_slope_multiplier': cml_slope_multiplier_slider,
    'x_min': x_min_slider,
    'x_max': x_max_slider,
    'y_min': y_min_slider,
    'y_max': y_max_slider,
    'cml_alpha': cml_alpha_slider
})

# Display the controls and the plot side by side
ui = HBox([controls, out])

display(ui)
